In [1]:
import sys
import os
import json
import pandas
import numpy
import optparse

from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from collections import OrderedDict
from sklearn.model_selection import train_test_split 
from urllib.parse import urlparse, parse_qs

In [2]:
data = []

def file_to_dataframe(filepath, datatype):
    with open(filepath, 'r') as fd:
        lines = fd.readlines()
        request ={}
        for line in lines:
            if len(line.strip()) == 0:
                continue
            elif line.startswith('GET') or line.startswith('POST') or line.startswith('PUT'):
                if request:
                    data.append(request)
                request ={}
                request['Data-Type'] = datatype
                line = line.split()
                request['Http-Type'] = line[0]
                parsed_url = urlparse(line[1])
                request['Http-Path'] = parsed_url.path
                request['Http-Query'] = parsed_url.query
            elif ' ' in line:
                line = line.split(':')
                key = line[0]
                value = ':'.join(line[1:]).strip()
                request[key] = value
            else:
                request['Http-Query'] = line

In [3]:
file_to_dataframe('normalTrafficTraining.txt', 0)
file_to_dataframe('anomalousTrafficTest.txt', 1)

In [4]:
df = pandas.DataFrame(data=data)
df.fillna('', inplace=True)

In [5]:
df = df.drop(['Content-Length', 'Cache-control'], axis=1)

In [6]:
df_datatype = df.pop('Data-Type')
df['Data-Type'] = df_datatype

In [7]:
dataset = df.sample(frac=1).values
dataset

array([['GET', '/tienda1/miembros/imagenes/zarauz.jpg', '', ..., 'close',
        '', 0],
       ['GET', '/tienda1/miembros/salir.jsp', '', ..., 'close', '', 0],
       ['GET', '/tienda1/index.jsp', '', ..., 'close', '', 0],
       ...,
       ['POST', '/tienda1/publico/autenticar.jsp',
        'modo=entrar&login=perkins%27%2C%270%27%2C%270%27%2C%270%27%2C%270%27%29%3Bwaitfor+delay+%270%3A0%3A15%27%3B--&pwd=7ad524z&remember=off&B1=Entrar\n',
        ..., 'close', 'application/x-www-form-urlencoded', 1],
       ['GET', '/tienda1/publico/autenticar.jsp',
        'modo=entrar&login=chandras&pwd=h%2Bolgona&remember=%3F&B1=Entrar',
        ..., 'close', '', 1],
       ['GET', '/tienda1/miembros/fotos.jsp', '', ..., 'close', '', 0]],
      dtype=object)

In [8]:
len(dataset[0])

14

In [9]:
X = dataset[:,:14]
X

array([['GET', '/tienda1/miembros/imagenes/zarauz.jpg', '', ..., 'close',
        '', 0],
       ['GET', '/tienda1/miembros/salir.jsp', '', ..., 'close', '', 0],
       ['GET', '/tienda1/index.jsp', '', ..., 'close', '', 0],
       ...,
       ['POST', '/tienda1/publico/autenticar.jsp',
        'modo=entrar&login=perkins%27%2C%270%27%2C%270%27%2C%270%27%2C%270%27%29%3Bwaitfor+delay+%270%3A0%3A15%27%3B--&pwd=7ad524z&remember=off&B1=Entrar\n',
        ..., 'close', 'application/x-www-form-urlencoded', 1],
       ['GET', '/tienda1/publico/autenticar.jsp',
        'modo=entrar&login=chandras&pwd=h%2Bolgona&remember=%3F&B1=Entrar',
        ..., 'close', '', 1],
       ['GET', '/tienda1/miembros/fotos.jsp', '', ..., 'close', '', 0]],
      dtype=object)

In [10]:
Y = dataset[:,13]
Y

array([0, 0, 0, ..., 1, 1, 0], dtype=object)

In [11]:
tokenizer = Tokenizer(filters='\t\n', char_level=True, lower=False)
tokenizer.fit_on_texts(X)

In [12]:
tokenizer.word_index

{'': 1,
 'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko)': 2,
 'no-cache': 3,
 'x-gzip, x-deflate, gzip, deflate': 4,
 'utf-8, utf-8;q=0.5, *;q=0.5': 5,
 'en': 6,
 'close': 7,
 'text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5': 8,
 'localhost:8080': 9,
 'GET': 10,
 0: 11,
 1: 12,
 'application/x-www-form-urlencoded': 13,
 'POST': 14,
 '/tienda1/publico/anadir.jsp': 15,
 '/tienda1/publico/autenticar.jsp': 16,
 '/tienda1/publico/registro.jsp': 17,
 '/tienda1/miembros/editar.jsp': 18,
 '/tienda1/publico/pagar.jsp': 19,
 '/tienda1/publico/caracteristicas.jsp': 20,
 '/tienda1/publico/vaciar.jsp': 21,
 '/tienda1/publico/entrar.jsp': 22,
 'B2=Vaciar+carrito\n': 23,
 'errorMsg=Credenciales+incorrectas\n': 24,
 '/tienda1/miembros/imagenes/zarauz.jpg': 25,
 '/tienda1/miembros/salir.jsp': 26,
 '/tienda1/index.jsp': 27,
 '/tienda1/global/titulo.jsp': 28,
 '/tienda1/global/creditos.jsp': 29,
 '/tienda1/miembros/index.jsp':

In [13]:
# Extract and save word dictionary
word_dict_file = 'dict/word-dictionary.json'

if not os.path.exists(os.path.dirname(word_dict_file)):
    os.makedirs(os.path.dirname(word_dict_file))

with open(word_dict_file, 'w') as outfile:
    json.dump(tokenizer.word_index, outfile, ensure_ascii=False)

In [14]:
num_words = len(tokenizer.word_index)+1
num_words

86659

In [15]:
X = tokenizer.texts_to_sequences(X)
X

[[10, 25, 1, 2, 3, 8, 4, 5, 6, 9, 1936, 7, 1, 11],
 [10, 26, 1, 2, 3, 8, 4, 5, 6, 9, 1937, 7, 1, 11],
 [10, 27, 1, 2, 3, 8, 4, 5, 6, 9, 1938, 7, 1, 11],
 [10, 15, 1939, 2, 3, 8, 4, 5, 6, 9, 1940, 7, 1, 12],
 [14, 21, 107, 2, 3, 8, 4, 5, 6, 9, 1941, 7, 13, 12],
 [10, 28, 1, 2, 3, 8, 4, 5, 6, 9, 1942, 7, 1, 11],
 [14, 18, 1943, 2, 3, 8, 4, 5, 6, 9, 1944, 7, 13, 12],
 [10, 16, 1945, 2, 3, 8, 4, 5, 6, 9, 1946, 7, 1, 11],
 [14, 18, 1947, 2, 3, 8, 4, 5, 6, 9, 1948, 7, 13, 11],
 [10, 29, 1, 2, 3, 8, 4, 5, 6, 9, 1949, 7, 1, 11],
 [10, 17, 1950, 2, 3, 8, 4, 5, 6, 9, 1951, 7, 1, 12],
 [14, 20, 57, 2, 3, 8, 4, 5, 6, 9, 1952, 7, 13, 12],
 [10, 46, 1, 2, 3, 8, 4, 5, 6, 9, 1953, 7, 1, 11],
 [10, 21, 51, 2, 3, 8, 4, 5, 6, 9, 1954, 7, 1, 12],
 [14, 16, 1955, 2, 3, 8, 4, 5, 6, 9, 1956, 7, 13, 11],
 [10, 70, 1, 2, 3, 8, 4, 5, 6, 9, 1957, 7, 1, 12],
 [10, 20, 48, 2, 3, 8, 4, 5, 6, 9, 1958, 7, 1, 11],
 [10, 28, 1, 2, 3, 8, 4, 5, 6, 9, 1959, 7, 1, 11],
 [10, 30, 1, 2, 3, 8, 4, 5, 6, 9, 1960, 7, 1, 11],
 [1

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=23)

In [17]:
X_train=numpy.asarray(X_train).astype(numpy.int)
X_test=numpy.asarray(X_test).astype(numpy.int)

y_train=numpy.asarray(y_train).astype(numpy.int)
y_test=numpy.asarray(y_test).astype(numpy.int)

In [18]:
tb_callback = TensorBoard(log_dir='./logs', embeddings_freq=1)

In [19]:
model = Sequential()
model.add(Embedding(num_words, 32, input_length=14))
model.add(Dropout(0.5))
model.add(LSTM(64, recurrent_dropout=0.5))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(numpy.array(X_train), numpy.array(y_train), validation_split=0.25, epochs=3, batch_size=128, callbacks=[tb_callback])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 14, 32)            2773088   
_________________________________________________________________
dropout (Dropout)            (None, 14, 32)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 2,797,985
Trainable params: 2,797,985
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
287/287 [==============================] - 10s 29ms/step - loss: 0.3564 - accuracy: 0.8047 - val_

In [20]:
score, acc = model.evaluate(numpy.array(X_test), numpy.array(y_test), verbose=1, batch_size=128)

96/96 [==============================] - 0s 3ms/step - loss: 1.6419e-05 - accuracy: 1.0000


In [21]:
print("Model Accuracy: {:0.2f}%".format(acc * 100))

Model Accuracy: 100.00%


In [22]:
predictions = model.predict(X_test)

In [23]:
print("Actual Estimated")
for i in range(0, len(y_test)):
    print(y_test[i], format(predictions[i][0], '0.5f'))

Actual Estimated
0 0.00000
0 0.00000
0 0.00000
1 0.99998
0 0.00010
0 0.00000
0 0.00000
0 0.00000
1 0.99998
0 0.00000
0 0.00000
1 0.99999
1 0.99998
1 0.99999
1 0.99999
1 0.99999
1 0.99999
0 0.00008
1 0.99999
0 0.00000
1 0.99998
0 0.00000
1 0.99998
0 0.00000
0 0.00000
1 0.99999
0 0.00000
0 0.00000
1 0.99998
0 0.00000
1 0.99998
1 0.99998
0 0.00000
1 0.99999
0 0.00000
0 0.00000
1 0.99998
0 0.00000
1 0.99999
0 0.00008
1 0.99998
1 0.99998
1 0.99999
0 0.00000
1 0.99998
0 0.00000
0 0.00000
1 0.99999
1 0.99998
1 0.99998
1 0.99998
1 0.99998
0 0.00000
0 0.00000
0 0.00000
1 0.99999
1 0.99999
0 0.00000
1 0.99999
0 0.00000
0 0.00000
0 0.00000
0 0.00008
0 0.00000
0 0.00000
0 0.00000
0 0.00000
1 0.99998
0 0.00000
1 0.99998
1 0.99999
1 0.99999
0 0.00007
1 0.99998
0 0.00007
0 0.00000
1 0.99998
0 0.00000
1 0.99998
0 0.00000
0 0.00008
1 0.99999
0 0.00000
1 0.99998
1 0.99999
1 0.99999
0 0.00006
1 0.99998
1 0.99999
1 0.99998
1 0.99998
1 0.99998
0 0.00006
0 0.00000
0 0.00000
0 0.00000
0 0.00000
1 0.99998
0 0

0 0.00000
1 0.99998
0 0.00000
0 0.00000
1 0.99998
1 0.99999
0 0.00008
1 0.99998
0 0.00000
0 0.00000
1 0.99998
0 0.00000
0 0.00006
1 0.99998
1 0.99998
1 0.99998
0 0.00000
1 0.99999
0 0.00000
1 0.99998
0 0.00001
1 0.99999
1 0.99998
0 0.00000
0 0.00000
1 0.99999
0 0.00000
0 0.00000
1 0.99999
0 0.00000
1 0.99998
0 0.00000
0 0.00000
0 0.00004
0 0.00000
1 0.99999
0 0.00000
1 0.99999
1 0.99998
0 0.00000
1 0.99999
0 0.00000
0 0.00008
1 0.99999
1 0.99999
0 0.00000
0 0.00000
0 0.00007
0 0.00005
0 0.00000
0 0.00008
1 0.99999
0 0.00005
0 0.00000
1 0.99999
1 0.99998
0 0.00000
1 0.99998
1 0.99999
1 0.99998
1 0.99998
0 0.00000
0 0.00000
0 0.00008
0 0.00000
1 0.99999
1 0.99999
0 0.00000
1 0.99997
0 0.00000
0 0.00000
0 0.00000
0 0.00000
0 0.00007
1 0.99998
0 0.00000
0 0.00000
1 0.99998
1 0.99999
1 0.99999
1 0.99998
1 0.99999
0 0.00000
0 0.00000
0 0.00005
0 0.00000
0 0.00000
1 0.99998
0 0.00000
1 0.99999
0 0.00000
1 0.99998
0 0.00000
0 0.00005
1 0.99998
0 0.00000
0 0.00000
0 0.00000
1 0.99999
1 0.99999


0 0.00000
0 0.00000
0 0.00005
0 0.00005
1 0.99999
1 0.99998
0 0.00000
0 0.00000
0 0.00000
0 0.00004
0 0.00000
0 0.00000
1 0.99999
0 0.00005
1 0.99998
1 0.99999
1 0.99999
1 0.99998
0 0.00000
0 0.00000
0 0.00008
1 0.99998
0 0.00000
0 0.00000
0 0.00000
0 0.00000
0 0.00000
1 0.99998
1 0.99998
0 0.00000
0 0.00000
1 0.99998
1 0.99999
1 0.99999
0 0.00000
0 0.00005
1 0.99998
1 0.99999
0 0.00007
0 0.00000
1 0.99998
0 0.00000
1 0.99999
1 0.99998
0 0.00006
0 0.00000
0 0.00006
0 0.00000
0 0.00007
1 0.99999
0 0.00000
1 0.99998
1 0.99998
0 0.00000
1 0.99998
1 0.99998
0 0.00000
0 0.00010
0 0.00000
0 0.00000
1 0.99999
0 0.00005
0 0.00000
1 0.99998
1 0.99998
1 0.99998
1 0.99998
0 0.00007
1 0.99998
1 0.99999
1 0.99998
0 0.00000
0 0.00000
0 0.00007
0 0.00000
0 0.00000
0 0.00000
1 0.99998
1 0.99998
1 0.99998
0 0.00005
1 0.99999
0 0.00005
0 0.00000
1 0.99998
0 0.00000
1 0.99998
1 0.99999
0 0.00000
1 0.99999
1 0.99998
1 0.99998
0 0.00000
0 0.00000
0 0.00000
0 0.00000
1 0.99999
1 0.99998
1 0.99999
1 0.99998


1 0.99998
0 0.00000
0 0.00000
1 0.99998
1 0.99999
0 0.00000
1 0.99998
1 0.99998
1 0.99998
1 0.99998
0 0.00005
1 0.99998
1 0.99998
1 0.99998
0 0.00000
1 0.99999
1 0.99998
0 0.00000
0 0.00007
0 0.00000
1 0.99998
1 0.99998
0 0.00006
1 0.99999
0 0.00000
0 0.00000
0 0.00007
0 0.00007
0 0.00009
0 0.00003
0 0.00006
1 0.99998
0 0.00007
0 0.00000
0 0.00008
1 0.99999
1 0.99998
1 0.99998
0 0.00000
1 0.99998
0 0.00000
1 0.99999
0 0.00000
0 0.00000
0 0.00000
1 0.99998
0 0.00000
0 0.00000
0 0.00000
0 0.00000
1 0.99999
1 0.99998
1 0.99998
0 0.00000
1 0.99999
1 0.99999
0 0.00000
1 0.99999
0 0.00000
0 0.00000
1 0.99999
0 0.00000
0 0.00004
0 0.00004
0 0.00000
0 0.00000
0 0.00000
0 0.00004
0 0.00000
1 0.99999
0 0.00000
0 0.00000
0 0.00000
0 0.00005
0 0.00000
0 0.00000
0 0.00005
0 0.00000
1 0.99998
1 0.99999
1 0.99999
1 0.99999
0 0.00000
0 0.00000
0 0.00007
0 0.00000
0 0.00000
0 0.00000
0 0.00006
1 0.99998
0 0.00000
0 0.00000
1 0.99999
1 0.99999
0 0.00005
1 0.99998
0 0.00000
1 0.99999
1 0.99999
1 0.99998


0 0.00003
0 0.00012
0 0.00000
0 0.00011
1 0.99998
0 0.00000
1 0.99999
0 0.00012
1 0.99998
1 0.99998
0 0.00000
0 0.00000
0 0.00000
0 0.00000
0 0.00000
1 0.99998
0 0.00006
0 0.00000
0 0.00000
1 0.99998
0 0.00000
0 0.00000
1 0.99999
1 0.99998
0 0.00000
0 0.00010
0 0.00004
0 0.00000
0 0.00002
0 0.00000
0 0.00004
0 0.00000
0 0.00009
0 0.00000
1 0.99998
0 0.00000
1 0.99999
0 0.00000
1 0.99999
0 0.00000
0 0.00000
0 0.00000
0 0.00000
0 0.00000
1 0.99999
0 0.00000
0 0.00000
0 0.00000
1 0.99999
0 0.00000
1 0.99999
1 0.99998
0 0.00000
1 0.99999
0 0.00000
0 0.00000
1 0.99999
1 0.99998
0 0.00004
0 0.00000
0 0.00000
0 0.00000
0 0.00006
0 0.00007
1 0.99998
0 0.00000
0 0.00000
1 0.99998
1 0.99999
0 0.00000
0 0.00000
0 0.00000
0 0.00000
0 0.00000
1 0.99998
0 0.00000
0 0.00000
0 0.00004
0 0.00000
0 0.00008
0 0.00000
0 0.00000
0 0.00000
0 0.00005
0 0.00000
1 0.99999
1 0.99998
1 0.99998
1 0.99999
0 0.00000
1 0.99998
1 0.99998
0 0.00000
0 0.00000
0 0.00005
1 0.99999
0 0.00000
0 0.00005
1 0.99998
0 0.00005


0 0.00000
0 0.00004
1 0.99998
0 0.00000
0 0.00008
0 0.00004
0 0.00000
1 0.99998
1 0.99999
0 0.00000
0 0.00000
0 0.00000
0 0.00000
1 0.99999
0 0.00000
0 0.00000
0 0.00000
1 0.99998
0 0.00004
1 0.99999
0 0.00008
1 0.99999
1 0.99998
1 0.99998
1 0.99999
0 0.00000
0 0.00000
1 0.99999
1 0.99998
0 0.00000
0 0.00000
0 0.00005
1 0.99998
1 0.99999
0 0.00009
1 0.99999
1 0.99999
0 0.00007
1 0.99998
0 0.00000
0 0.00000
0 0.00006
1 0.99998
1 0.99998
0 0.00000
0 0.00000
0 0.00004
1 0.99999
0 0.00000
1 0.99998
0 0.00000
1 0.99999
1 0.99999
1 0.99998
0 0.00000
0 0.00009
0 0.00000
0 0.00008
1 0.99998
1 0.99999
0 0.00000
0 0.00000
1 0.99998
0 0.00000
1 0.99998
0 0.00000
0 0.00000
0 0.00000
0 0.00000
0 0.00007
1 0.99999
0 0.00000
0 0.00000
0 0.00000
1 0.99999
0 0.00010
0 0.00000
1 0.99999
1 0.99998
1 0.99998
0 0.00000
0 0.00000
0 0.00000
0 0.00000
0 0.00000
1 0.99998
0 0.00000
0 0.00000
1 0.99999
1 0.99999
0 0.00000
1 0.99998
0 0.00000
1 0.99999
0 0.00004
1 0.99999
0 0.00007
0 0.00000
0 0.00000
0 0.00000


0 0.00000
0 0.00000
0 0.00006
0 0.00000
1 0.99999
1 0.99999
0 0.00008
0 0.00000
0 0.00009
0 0.00000
0 0.00000
1 0.99998
1 0.99998
1 0.99998
0 0.00003
0 0.00000
0 0.00008
0 0.00000
1 0.99998
1 0.99998
0 0.00000
1 0.99999
0 0.00000
0 0.00000
0 0.00000
0 0.00003
0 0.00005
1 0.99998
0 0.00008
1 0.99998
0 0.00007
1 0.99998
1 0.99999
1 0.99999
1 0.99999
1 0.99998
0 0.00000
0 0.00007
0 0.00000
0 0.00000
0 0.00000
0 0.00000
0 0.00011
0 0.00000
0 0.00007
0 0.00000
0 0.00000
1 0.99998
0 0.00000
0 0.00000
1 0.99998
0 0.00000
1 0.99999
0 0.00000
0 0.00000
0 0.00000
0 0.00000
1 0.99999
0 0.00000
0 0.00000
1 0.99998
1 0.99999
0 0.00000
0 0.00009
0 0.00000
0 0.00000
0 0.00006
0 0.00000
0 0.00000
0 0.00000
1 0.99998
1 0.99999
0 0.00000
0 0.00007
0 0.00000
1 0.99999
1 0.99998
0 0.00000
1 0.99998
0 0.00000
0 0.00004
0 0.00006
0 0.00000
1 0.99998
1 0.99998
0 0.00000
0 0.00004
0 0.00000
0 0.00000
1 0.99998
0 0.00000
1 0.99998
0 0.00003
0 0.00005
1 0.99998
0 0.00000
1 0.99998
0 0.00000
0 0.00008
0 0.00007


0 0.00000
0 0.00010
0 0.00000
0 0.00000
0 0.00005
1 0.99998
1 0.99999
1 0.99998
0 0.00000
1 0.99998
0 0.00000
0 0.00000
0 0.00000
1 0.99999
1 0.99999
1 0.99998
1 0.99999
0 0.00000
0 0.00000
1 0.99999
0 0.00000
0 0.00000
0 0.00000
1 0.99999
0 0.00010
0 0.00005
0 0.00000
1 0.99998
1 0.99998
1 0.99998
0 0.00004
1 0.99998
1 0.99999
1 0.99998
0 0.00000
0 0.00005
1 0.99998
0 0.00000
1 0.99998
0 0.00009
1 0.99998
1 0.99998
1 0.99998
1 0.99998
1 0.99999
1 0.99998
1 0.99998
1 0.99999
0 0.00000
1 0.99999
0 0.00000
1 0.99998
0 0.00000
1 0.99998
0 0.00000
1 0.99998
0 0.00000
0 0.00000
1 0.99998
0 0.00000
1 0.99998
1 0.99998
0 0.00004
1 0.99998
0 0.00000
0 0.00000
1 0.99998
0 0.00000
0 0.00000
1 0.99998
0 0.00004
0 0.00000
0 0.00007
1 0.99998
1 0.99998
1 0.99999
0 0.00000
0 0.00000
1 0.99999
1 0.99998
1 0.99998
0 0.00000
0 0.00007
1 0.99998
1 0.99998
1 0.99998
0 0.00007
0 0.00000
1 0.99999
0 0.00000
1 0.99998
1 0.99998
0 0.00000
1 0.99998
0 0.00000
0 0.00000
0 0.00010
1 0.99998
1 0.99999
0 0.00007


0 0.00000
0 0.00005
0 0.00000
1 0.99998
0 0.00000
0 0.00000
1 0.99998
0 0.00000
0 0.00000
0 0.00000
0 0.00000
0 0.00000
0 0.00000
0 0.00000
1 0.99999
1 0.99999
0 0.00000
1 0.99999
0 0.00000
0 0.00006
0 0.00006
0 0.00006
0 0.00000
1 0.99998
0 0.00000
1 0.99998
1 0.99998
0 0.00000
1 0.99999
1 0.99998
1 0.99999
0 0.00006
0 0.00000
0 0.00000
0 0.00005
0 0.00000
0 0.00000
0 0.00000
1 0.99998
0 0.00000
0 0.00000
0 0.00000
0 0.00007
1 0.99999
0 0.00000
0 0.00006
0 0.00000
1 0.99998
0 0.00000
0 0.00000
1 0.99999
0 0.00000
0 0.00007
0 0.00004
0 0.00000
0 0.00006
0 0.00000
0 0.00005
1 0.99998
0 0.00000
1 0.99998
0 0.00000
0 0.00000
0 0.00000
1 0.99998
0 0.00004
0 0.00000
0 0.00000
0 0.00000
1 0.99998
1 0.99999
0 0.00000
0 0.00007
1 0.99999
0 0.00000
1 0.99999
0 0.00000
1 0.99998
0 0.00000
0 0.00000
0 0.00006
0 0.00006
1 0.99998
0 0.00000
1 0.99999
0 0.00000
1 0.99998
1 0.99999
0 0.00000
0 0.00000
0 0.00005
1 0.99998
0 0.00000
0 0.00000
0 0.00000
0 0.00000
1 0.99998
1 0.99999
1 0.99998
1 0.99999
